In [26]:
import pandas as pd
import dotenv
dotenv.load_dotenv()

True

# Reading Data
Read in news data, drop duplicates then parse published date into Timestamp object

In [27]:
import datetime
import dateutil.parser
import os
if not os.path.isfile(".data/processed_news.pickle"):
    news = pd.read_pickle(".data/organized_news.pickle")
    news.drop([news.columns[0]], axis=1, inplace=True)
    news.drop_duplicates(["url"])
    news = news[news["fetched_body"] != ""]
    news['week'] = news['published date'].apply(lambda x: x.isocalendar()[1])
    news['year'] = news['published date'].apply(lambda x:  x.isocalendar()[0])
    sampled_news = news
else:
    sampled_news = pd.read_pickle(".data/processed_news.pickle")
sampled_news

,description,published date,url,publisher,fetched_title,fetched_body,Topic,week,year,gpt_results
0,Tech Concentration in the S&P 500 is Highest S...,2017-06-20 07:00:00+00:00,https://news.google.com/rss/articles/CBMiUWh0d...,"{'href': 'https://www.investopedia.com', 'titl...",Tech Concentration in the S&P 500 is Highest S...,It has been a good year for the technology sec...,AAPL,25,2017,Summary: The tech sector has been performing e...
1,'A watershed moment': Tech execs react to Uber...,2017-06-21 07:00:00+00:00,https://news.google.com/rss/articles/CBMiZWh0d...,"{'href': 'https://finance.yahoo.com', 'title':...",‘A watershed moment’: Tech execs react to Uber...,Former Uber CEO and co-founder Travis Kalanick...,AAPL,25,2017,Summary: Travis Kalanick has officially steppe...
2,Tim Cook offloads more than $43 million in AAP...,2017-08-28 07:00:00+00:00,https://news.google.com/rss/articles/CBMiVGh0d...,"{'href': 'https://9to5mac.com', 'title': '9to5...",Tim Cook offloads more than $43 million in AAP...,Apple CEO Tim Cook has sold some $43 million w...,AAPL,35,2017,Summary: Apple CEO Tim Cook has sold $43 milli...
3,Even an Apple store can't prevent the death of...,2017-09-02 07:00:00+00:00,https://news.google.com/rss/articles/CBMiTmh0d...,"{'href': 'https://qz.com', 'title': 'Quartz'}",Even an Apple store can’t prevent the death of...,"The American mall is dying, and not even Apple...",AAPL,35,2017,Summary: Despite the decline of American malls...
4,The Apple TV is not the 'future of TV' Apple p...,2017-08-29 07:00:00+00:00,https://news.google.com/rss/articles/CBMiTGh0d...,"{'href': 'https://www.businessinsider.com', 't...",The Apple TV is a mess — and hardly the 'futur...,I’ve owned the fourth-generation Apple TV for ...,AAPL,35,2017,Summary: The current Apple TV hardware is capa...
...,...,...,...,...,...,...,...,...,...,...
5025,Longer Term Outlooks Using NDX Options Nasdaq,2023-05-22 07:00:00+00:00,https://news.google.com/rss/articles/CBMiRmh0d...,"{'href': 'https://www.nasdaq.com', 'title': 'N...",Longer Term Outlooks Using NDX Options,We know all the rage is about short-dated opti...,^NDX,21,2023,
5026,Is the NASDAQ 100 Close to Topping out? | inve...,2023-05-25 07:00:00+00:00,https://news.google.com/rss/articles/CBMiU2h0d...,"{'href': 'https://www.investing.com', 'title':...",Is the NASDAQ 100 Close to Topping out?,NDX -1.15% Add to/Remove from Watchlist By us...,^NDX,21,2023,
5027,Indexed Finance (NDX) Receives a Very Bullish ...,2023-05-26 07:00:00+00:00,https://news.google.com/rss/articles/CBMiiQFod...,"{'href': 'https://www.investorsobserver.com', ...",Indexed Finance (NDX) Receives a Very Bullish ...,Indexed Finance has a Very Bullish sentiment r...,^NDX,21,2023,
5028,NDX Option Numbers to Know Before Tomorrow's C...,2023-05-09 07:00:00+00:00,https://news.google.com/rss/articles/CBMiT2h0d...,"{'href': 'https://www.nasdaq.com', 'title': 'N...",NDX Option Numbers to Know Before Tomorrow’s CPI,Over the past year and a half or so the invest...,^NDX,19,2023,


In [28]:

sampled_news["fetched_body"] = sampled_news["fetched_body"].convert_dtypes(convert_string=True)
sampled_news = sampled_news.dropna(inplace=False)
if "gpt_results" not in sampled_news:
    sampled_news["gpt_results"] = ""

In [29]:
sampled_news["fetched_body"].apply(lambda x: x.upper())

0       IT HAS BEEN A GOOD YEAR FOR THE TECHNOLOGY SEC...
1       FORMER UBER CEO AND CO-FOUNDER TRAVIS KALANICK...
2       APPLE CEO TIM COOK HAS SOLD SOME $43 MILLION W...
3       THE AMERICAN MALL IS DYING, AND NOT EVEN APPLE...
4       I’VE OWNED THE FOURTH-GENERATION APPLE TV FOR ...
                              ...                        
5025    WE KNOW ALL THE RAGE IS ABOUT SHORT-DATED OPTI...
5026    NDX -1.15% ADD TO/REMOVE FROM WATCHLIST\n\nBY ...
5027    INDEXED FINANCE HAS A VERY BULLISH SENTIMENT R...
5028    OVER THE PAST YEAR AND A HALF OR SO THE INVEST...
5030    NASDAQ 100 & USD/JPY FORECAST:\n\nNASDAQ 100 G...
Name: fetched_body, Length: 4717, dtype: object

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import langchain
from openai import RateLimitError

In [31]:
gpt = ChatOpenAI(model="gpt-3.5-turbo")

# Run Sample through GPT (Remote Large Language Model)

In [32]:
news_summary_prompt = ChatPromptTemplate.from_template("Summarize the following noisy news data and identify keywords of the news relevant to a Apple stock analyst. Provide a formatted answer as such Summary: ..., Keywords: ... News: {news}")
news_summary_chain = news_summary_prompt | gpt | StrOutputParser()
gpt_results = news_summary_chain.invoke({"news": sampled_news["fetched_body"].iloc[-1]})
print(gpt_results)

Summary: The Nasdaq 100 rallied after U.S. inflation data came in lower than expected, while the USD/JPY fell due to lower U.S. Treasury yields following an encouraging CPI report. The article discusses key price levels to watch on both charts and emphasizes the importance of a healthy economy for stock performance.

Keywords: Nasdaq 100, USD/JPY, technical resistance, U.S. inflation data, U.S. Treasury yields, CPI report, economic outlook, technical analysis, support, resistance, monetary policy easing, rate-sensitive Japanese yen.


Using the delay of the response from the remote LLM as a natural way of rate limiting the system to avoid running into gpt rate cap

In [33]:
def batched(iter, batch_size):
    temp = []
    for i in iter:
        temp.append(i)
        if len(temp) == batch_size:
            yield temp
            temp = []
    yield temp

In [25]:
import time
for article_index in (sampled_news[sampled_news["gpt_results"] == ""].index):
    prompt_article = {"news": sampled_news["fetched_body"].loc[article_index][:15000*4]}
    for i in range(5):
        try:
            result = news_summary_chain.invoke(prompt_article)
            sampled_news.loc[article_index, "gpt_results"] = result
            print(f"{article_index}: {result}")
            sampled_news.to_pickle(".data/processed_news.pickle")
            break
        except RateLimitError:
            time.sleep(20)
            continue
        except Exception as e:
            print(e)
            assert False

Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 19805 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


AssertionError: 

In [110]:
summaries = [["AAPL","^NDX","^GSPC"], ["GOOGL","^NDX","^GSPC"], ["AMZN","^NDX","^GSPC"], ["MSFT","^NDX","^GSPC"]]
for summary in summaries:
    

,description,published date,url,publisher,fetched_title,fetched_body,Topic,week,year,gpt_results
0,Tech Concentration in the S&P 500 is Highest S...,2017-06-20 07:00:00+00:00,https://news.google.com/rss/articles/CBMiUWh0d...,"{'href': 'https://www.investopedia.com', 'titl...",Tech Concentration in the S&P 500 is Highest S...,It has been a good year for the technology sec...,AAPL,25,2017,Summary: The tech sector has been performing e...
1,'A watershed moment': Tech execs react to Uber...,2017-06-21 07:00:00+00:00,https://news.google.com/rss/articles/CBMiZWh0d...,"{'href': 'https://finance.yahoo.com', 'title':...",‘A watershed moment’: Tech execs react to Uber...,Former Uber CEO and co-founder Travis Kalanick...,AAPL,25,2017,Summary: Travis Kalanick has officially steppe...
2,Tim Cook offloads more than $43 million in AAP...,2017-08-28 07:00:00+00:00,https://news.google.com/rss/articles/CBMiVGh0d...,"{'href': 'https://9to5mac.com', 'title': '9to5...",Tim Cook offloads more than $43 million in AAP...,Apple CEO Tim Cook has sold some $43 million w...,AAPL,35,2017,Summary: Apple CEO Tim Cook has sold $43 milli...
3,Even an Apple store can't prevent the death of...,2017-09-02 07:00:00+00:00,https://news.google.com/rss/articles/CBMiTmh0d...,"{'href': 'https://qz.com', 'title': 'Quartz'}",Even an Apple store can’t prevent the death of...,"The American mall is dying, and not even Apple...",AAPL,35,2017,Summary: Despite the decline of American malls...
4,The Apple TV is not the 'future of TV' Apple p...,2017-08-29 07:00:00+00:00,https://news.google.com/rss/articles/CBMiTGh0d...,"{'href': 'https://www.businessinsider.com', 't...",The Apple TV is a mess — and hardly the 'futur...,I’ve owned the fourth-generation Apple TV for ...,AAPL,35,2017,Summary: The current Apple TV hardware is capa...
...,...,...,...,...,...,...,...,...,...,...
5025,Longer Term Outlooks Using NDX Options Nasdaq,2023-05-22 07:00:00+00:00,https://news.google.com/rss/articles/CBMiRmh0d...,"{'href': 'https://www.nasdaq.com', 'title': 'N...",Longer Term Outlooks Using NDX Options,We know all the rage is about short-dated opti...,^NDX,21,2023,
5026,Is the NASDAQ 100 Close to Topping out? | inve...,2023-05-25 07:00:00+00:00,https://news.google.com/rss/articles/CBMiU2h0d...,"{'href': 'https://www.investing.com', 'title':...",Is the NASDAQ 100 Close to Topping out?,NDX -1.15% Add to/Remove from Watchlist By us...,^NDX,21,2023,
5027,Indexed Finance (NDX) Receives a Very Bullish ...,2023-05-26 07:00:00+00:00,https://news.google.com/rss/articles/CBMiiQFod...,"{'href': 'https://www.investorsobserver.com', ...",Indexed Finance (NDX) Receives a Very Bullish ...,Indexed Finance has a Very Bullish sentiment r...,^NDX,21,2023,
5028,NDX Option Numbers to Know Before Tomorrow's C...,2023-05-09 07:00:00+00:00,https://news.google.com/rss/articles/CBMiT2h0d...,"{'href': 'https://www.nasdaq.com', 'title': 'N...",NDX Option Numbers to Know Before Tomorrow’s CPI,Over the past year and a half or so the invest...,^NDX,19,2023,


In [16]:
pd.pivot_table(sampled_news, values=["gpt_results"], index = ["year", "week", "Topic"], aggfunc="sum")

gpt_results
year week Topic                                                   
2017 23   GOOGL                                                   
     24   AAPL   Summary: Goldman Sachs predicts tech sector st...
          AMZN   Summary: The Amazon rainforest is at risk due ...
          GOOGL  Summary: Microsoft has entered the laptop mark...
          MSFT   Summary: Goldman Sachs predicts the end of the...
...                                                            ...
2023 22   AMZN                                                    
          GOOGL                                                   
          MSFT                                                    
          ^GSPC                                                   
          ^NDX                                                    

[1477 rows x 1 columns]